In [ ]:
import airflow
from airflow.operators.python_operator import BranchPythonOperator, PythonOperator
from airflow.operators.dummy_operator import DummyOperator
from airflow.models import DAG
import random

## setup the DAG w/ some sane defaults
args = {
    'owner': 'airflow',
    'start_date': airflow.utils.dates.days_ago(1)
}

dag = DAG(
    dag_id='example_dag_for_prefect',
    default_args=args,
    schedule_interval="@once")

## add a dummy task to run first
run_this_first = DummyOperator(task_id='run_this_first', dag=dag)

## add a conditional which will randomly choose to follow one of `branch_a` or `branch_b`
options = ['branch_a', 'branch_b']

branching = BranchPythonOperator(
    task_id='branching',
    python_callable=lambda: random.choice(options),
    dag=dag)

branching.set_upstream(run_this_first)

## add a dummy terminal task w/ a unique trigger
join = DummyOperator(
    task_id='join',
    trigger_rule='one_success',
    dag=dag
)


## will push an xcom with the value of "value"
def push_xcom(value, **kwargs):
    kwargs['ti'].xcom_push(key='value from push_xcom task', value=value)


for option in options:
    t = DummyOperator(task_id=option, dag=dag)
    t.set_upstream(branching)
    xcom_follow = PythonOperator(task_id='push_xcom_for_' + option, 
                                 dag=dag, 
                                 python_callable=lambda value=option, **kwargs: push_xcom(value, **kwargs),
                                 provide_context=True)
    t.set_downstream(xcom_follow)
    xcom_follow.set_downstream(join)


In [ ]:
from prefect.utilities.airflow_utils import AirFlow


flow = AirFlow(dag_id='example_dag_for_prefect')
flow.visualize()


In [ ]:
from prefect import task, triggers


@task(trigger=triggers.always_run, skip_on_upstream_skip=False) # discuss triggers and default skip cascading
def print_value(x):
    print(x)
    

xcom_a = flow.get_tasks(name='push_xcom_for_branch_a')[0]
xcom_b = flow.get_tasks(name='push_xcom_for_branch_b')[0]


with flow:
    # the value of the xcom that each push_xcom task created will be passed in as a function argument
    print_value(xcom_a) 
    print_value(xcom_b)
    
    
flow.visualize()


In [ ]:
from prefect.utilities.debug import raise_on_exception

flow_state = flow.run(execution_date='2018-09-20', return_tasks=flow.tasks)
#    branch_b
#    None
flow.visualize(flow_state=flow_state)
